In [1]:
import pandas as pd
import numpy as np

In [4]:
#import Tempreture data
df = pd.read_csv('../data/clean_SSTEMP_sensor.csv')

In [6]:
df.head

<bound method NDFrame.head of           State                 Time         Device Unit
0     13.000000  2024-09-12 01:04:03  SSTEMP_sensor    C
1     12.900000  2024-09-12 01:04:28  SSTEMP_sensor    C
2     12.900000  2024-09-12 01:04:53  SSTEMP_sensor    C
3     12.900000  2024-09-12 01:05:18  SSTEMP_sensor    C
4     12.900000  2024-09-12 01:05:43  SSTEMP_sensor    C
...         ...                  ...            ...  ...
6211  13.200674  2024-09-13 21:13:01  SSTEMP_sensor    C
6212  13.160405  2024-09-13 21:13:26  SSTEMP_sensor    C
6213  13.136243  2024-09-13 21:13:51  SSTEMP_sensor    C
6214  13.121746  2024-09-13 21:14:16  SSTEMP_sensor    C
6215  13.100000  2024-09-13 21:14:41  SSTEMP_sensor    C

[6216 rows x 4 columns]>